# Jupyter Notebook Gruppe Tesla/Musk

Teammitglieder: Falk Jaresch, Michael Foeger, Paula Engelberg, Christoph Fink.

#### Rollenaufteilung:

- Datenrecherche und Ideenfindung - Gruppe
- Suche der passenden Datenquellen - Gruppe
- Konvertierung von CSV in RDF - Foeger
- Kombination der Datenquellen und Triple Store Implementation - Engelberg
- Anfragen an die Datenquellen - Fink
- Erstellen einer Jekyll Seite - Jaresch

- Extra: Datenbearbeitung in OpenRefine (Datum) - Fink

#### Probleme:
Beim Erstellen der Queries für Aktien-Tagespreise an den bestimmten Tagen war es schwierig per Syntax die Formatierung der ersten Spalte (Datum) anzupassen.
Erst nach viel Recherche und Rücksprache mit Herrn Arndt war es klar, dass der einfachste Weg bei beiden Datenquellen eine einheitliche Formattierung des Datums zu erhalten (xsd:dateTime) war über OpenRefine.
Wir haben die Tesla-Stock CSV Datei in OpenRefine reingeladen und dann per Java Befehl "value.toDate('mm/dd/yyyy').toString('YYYY-MM-DD')" das Datum von einem amerikanischen Format auf das korrekte übersetzt. Danach musste nur noch die CSV Datei erneut in ein RDF File transformiert werden.

#### Reflexion der Techniken in Bezug auf den Technologiestack
-

#### Einverständnis zur Weitergabe erteilt




## Projekt Ausarbeitung

Datenquellen:

- Elon Musk Tweets: https://www.kaggle.com/kulgen/elon-musks-tweets

- Tesla Stock Price: https://www.kaggle.com/rpaguirre/tesla-stock-price



## Import der Datenquelle

### Vorgehen mit Docker

Erstellung eines Projektordners auf dem lokalen Rechner. Hier wurden in weiterer Folge die beiden Dateien mit der Endung .ttl hineinkopiert. Im nächsten Schritt musste der Quit Store als Docker Image heruntergeladen werden. Das Image wurde durch Ausführen des Kommandos: `docker run --name quit -it --rm -p 8082:8080 -v "$PWD":/data aksw/quitstore:0.24.3` im Terminal gestartet. Wobei mit `-v "$PWD":/data` das lokale Repository mit dem /data Ordner im Docker verlinkt wurde. So hat Docker Zugriff auf die Dateien im lokalen Ordner. $PWD wird unter Unix Systemen verwendet und gibt den Pfad an, in welchem man sich gerade befinden → Present Working Directory. Kann auch durch die händische Eingabe des gewünschten Pfades ersetzt werden: /home/Documents… etc.


### Aufgetretene Probleme
Am anfang wusste ich nicht, wie ich meinen lokalen Ordner mit den .ttl Daten mit Docker verbinde. Die Verbindung hat mir gefehlt, da ich die Ordnerstruktur von Docker nicht kenne bzw. nicht verstanden habe. Nach einigen Stunden habe ich geswitched und versucht, den Quit Store über das Git Repository zu clonen. Dies hat nicht funktioniert und ich habe ständig die Fehlermeldung: `error: command ‘xcrun’ failed with exit status 1` bekommen. Nach der Fehlermeldung habe ich alle möglichen Programme upgedated, sowie einige Python Packages, die auf Stackoverflow angeführt waren heruntergeladen, leider ohne Erfolg. Ich bekam trotzdem immer wieder die oben genannte Fehlermeldung und zusätzlich noch: `fatal error: ‘git2.h’ file not found`. Auf der Suche nach Fehlerbehebung habe ich festgestellt, dass mir ein Phyton Developer Package fehlt, dieses aber scheinbar für Mac nicht existier bzw. schon installiert sein sollte.

Danach habe ich wieder zu Docker gewechselt.


### Arbeiten mit Docker Shell

Im nächsten Schritt wurde die Shell von Docker aufgemacht.

![title](Docker_Shell.png)

Hier musste so lange hinauf navigiert werden, bis man zu der Übersicht kam, wo ein /data-Ordner zu sehen war. In dem /data-Ordner befanden sich dann die beiden .ttl-Dateien. Das war der Dateipfad, der oben mit `-v "$PWD":/data` angegeben wurde. Erst nach der Navigation in diesen Ordner konnte mittels `quit -t ~/data -p 8082` der Quit Store gestartet werden.

### Auftretende Probleme

Nachdem ich herausgefunden habe, wie man durch Docker navigiert, habe ich den Befehl aus der Moodle-Doku (unter weitere Tools) `quit-bin -t /repository/directory -p 8082` ausgeführt. Der Befehl `quit-bin` wurde jedoch nicht gefunden (siehe Bild unten). Das `/repository/directory` musst auch durch `~/data` ersetzt werden, da es mein Zielverzeichnis ist.

![](Docker_Fehlermeldung.png)

Der Quit Store greift bei Docker auf die Daten über /data zu und dann greift Docker auf den lokalen Ordner auf meinem Rechner mit den Daten zu.

In [1]:
from rdflib import Graph
from pandas import DataFrame

### Serialisierung: Parsen von Turtle zu ntriple

In [2]:
g = Graph()
g.parse("tesla.ttl", format="ttl")
g.serialize(destination='tesla.nt', format="ntriples")

In [3]:
g.parse("tweets.ttl", format="ttl")
g.serialize(destination='tweets.nt', format='ntriples')

Nachdem die beiden .ttl-Dateien serialisiert wurden, wurden sie in den Projektordner gelegt (siehe Abbildung unten).

![](Lokaler_Projektordner.png)

Danach wurde über das Terminal eine neue Datei mit dem Namen tesla.nt.graph angelegt und die URI http://example.org/resource/elonmusk/tesla/ eingefügt. Das gleiche wurde für tweets.nt.graph gemacht, der Inhalt ist jedoch: http://example.org/resource/elonmusk/tweet/

Alle neu erstellten Dateien wurden in Git commited und eine Query auf die Daten ist nun über den Quit Store möglich.


### Auftretende Probleme

Ich habe zuerst nur die tesla.ttl serialisiert und wollte schauen, ob die Abfrage über den Quit Store funktioniert. Zu diesem Zeitpunkt hatte ich noch die Daten aus der Übung im Ordner. Die Query hat immer nur die vier Datensätze aus der Übung angezeigt, obwohl die tesla.nt und tesla.nt.graph commited waren.
Nachdem ich unzählige W3C Dokumente durchforstet habe, habe ich zu meinem Problem keine Lösung gefunden.

#### Tipp von Natanael:
Meine Sparql-Query war falsch! Ich hatte folgendes in der Query stehen:
```SQL
SELECT ?subject ?predicate ?object
WHERE {?subject ?predicate ?object}
```
Hier muss aber noch der Graph angegeben werden:
```SQL
SELECT ?subject ?predicate ?object
WHERE { GRAPH <http://example.org/resource/elonmusk/tesla/>
          { ?subject ?predicate ?object} }
```
Die Query wurde ausgebessert, lieferte aber immer noch keine Ergebnisse.

#### Tipp von Natanael:
Man darf keine zwei gleichen graph.nt.graph-Dateien haben. Das heißt, der Name und der Inhalt dieser Dateien mpssen unterschiedlich sein. Danach habe ich die Dateien aus der Übung gelöscht und in die tesla.nt.graph-Datei, in welcher zuvor nur http://example.org/ stand mit http://example.org/resource/elonmusk/tesla/ ergänzt. Der Schrägstrich nach /tesla/ ist sehr wichtig, ohne funktioniert es nicht.

Danach konnte ich eine Query sowohl auf die Tweets als auch auf die Tesla Stock Daten durchführen und bekam auch Daten ausgegeben (siehe beide Abbildungen unten).


![](Abfrage_Tweets.png)

![](Abfrage_Tesla_Stock.png)

Nach diesem Schritt wurden alle Daten commited und in ein GitHub-Repository geladen und mit den Kollegen geteilt.

## Queries

### Query nach Aktienpreis (Trading Tag Ende)


In [15]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>       
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX sioc: <http://rdfs.org/sioc/ns#>
PREFIX dc:   <http://purl.org/dc/elements/1.1/>

SELECT ?date ?close

WHERE {
   ?tradingDay ex:close ?close ;
    ex:date ?date .

 }

ORDER BY DESC(?date)
""")

DataFrame(result, columns=result.vars)

,date,close
0,2017-01-31T09:00:00+00:00,251.929993
1,2017-01-30T09:00:00+00:00,250.630005
2,2017-01-28T09:00:00+00:00,249.990005
3,2017-01-27T09:00:00+00:00,246.229996
4,2017-01-27T09:00:00+00:00,252.949997
...,...,...
1684,2009-01-01T09:00:00+00:00,20.6
1685,2009-01-01T09:00:00+00:00,20.450001
1686,2009-01-01T09:00:00+00:00,21.41
1687,2009-01-01T09:00:00+00:00,21.959999


### Abfrage nach Tweet Inhalt

In [14]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>       
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX sioc: <http://rdfs.org/sioc/ns#>
PREFIX dc:   <http://purl.org/dc/elements/1.1/>

SELECT ?content

WHERE {
   <http://example.org/resource/elonmusk/tweet/Row47> <http://rdfs.org/sioc/ns#content> ?content 

 }

""")

DataFrame(result, columns=result.vars)

,content
0,@SmileSimplify Sorry about that. You're right....


### Filter Datum: Nur Kurs-Werte ab Jänner 2015

In [38]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>       
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX sioc: <http://rdfs.org/sioc/ns#>
PREFIX dc:   <http://purl.org/dc/elements/1.1/>

SELECT ?date ?close

WHERE {
   ?tradingDay ex:close ?close ;
    ex:date ?date .
    FILTER (?date > "2015-01-01T00:00:00Z"^^xsd:dateTime)
 }
 
ORDER BY DESC(?date)
""")

DataFrame(result, columns=result.vars)

,date,close
0,2017-01-31T09:00:00+00:00,251.929993
1,2017-01-30T09:00:00+00:00,250.630005
2,2017-01-28T09:00:00+00:00,249.990005
3,2017-01-27T09:00:00+00:00,246.229996
4,2017-01-27T09:00:00+00:00,252.949997
...,...,...
550,2015-01-01T09:00:00+00:00,216.5
551,2015-01-01T09:00:00+00:00,269.149994
552,2015-01-01T09:00:00+00:00,190.789993
553,2015-01-01T09:00:00+00:00,249.449997


### ASK: Is Elon Musk the Creator of the Tweets?

In [47]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>       
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX sioc: <http://rdfs.org/sioc/ns#>
PREFIX dc:   <http://purl.org/dc/elements/1.1/>

ASK
WHERE {
  <http://example.org/resource/elonmusk/tweet/Row562> <http://rdfs.org/sioc/ns#has_creator> "https://twitter.com/elonmusk" .
}
""")

DataFrame(result, columns=result.vars)

,0
0,True


### Bei DateTime die Uhrzeit entfernen


In [46]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>       
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX sioc: <http://rdfs.org/sioc/ns#>
PREFIX dc:   <http://purl.org/dc/elements/1.1/>

SELECT ?date ?close

WHERE {
   ?tradingDay ex:close ?close ;
    ex:date ?date .
    FILTER (?date "yyyy-mm-ddThh:mm:ss+hh:mm"^^xsd:dateTime -hh:mm:ss+hh:mm)
 }
 
ORDER BY DESC(?date)
""")

DataFrame(result, columns=result.vars)

ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found 'F'  (at char 388), (line:14, col:5)

### Kursabfrage an einem Datum

In [56]:
result = g.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>       
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX sioc: <http://rdfs.org/sioc/ns#>
PREFIX dc:   <http://purl.org/dc/elements/1.1/>

SELECT ?date ?close

WHERE {
   ?tradingDay ex:close ?close .
   ?FILTER(str(?date) = '2015-01-01T09:00:00+00:00')  .
}
 

""")

DataFrame(result, columns=result.vars)

ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found '?'  (at char 367), (line:13, col:4)